# Notebook to explore the tables created by dune_source using the default config in this repo
Before running this notebook, run the pipeline to create the tables in the duckdb database.:
```
python __init__.py
```
After running the pipeline, the duckdb database will be populated with the following tables:
- dex_volume
- y2k_price_data
- y2k_price_data_no_pk (no primary key), data is append-only
- custom_sql

Important: make sure to close the connection to the duckdb database after running the notebook, if you don't, the connection will be locked and you won't be able to run the pipeline again.

In [2]:
import duckdb
db = duckdb.connect("dune_source.duckdb")

In [3]:
# dex_volume
db.execute("select * from dune.dex_volume").fetchdf()

,project,_col1,usd_volume,_dlt_load_id,_dlt_id
0,0x-API,2025-02-11 00:00:00.000 UTC,1.587147e+07,1740275921.569736,9ktgI7W6ZNkqvg
1,0x-API,2025-02-18 00:00:00.000 UTC,1.927662e+07,1740275921.569736,R716Yb0AF556GA
2,1inch-LOP,2025-01-24 00:00:00.000 UTC,7.614095e+07,1740275921.569736,UJCG2DvH8amczg
3,agni,2025-01-28 00:00:00.000 UTC,4.685848e+06,1740275921.569736,y0iBt/xtoHlfrw
4,agni,2025-02-09 00:00:00.000 UTC,4.335497e+06,1740275921.569736,D2F0NApbB4Wlww
...,...,...,...,...,...
4108,zkswap_finance,2025-02-03 00:00:00.000 UTC,3.756933e+06,1740275921.569736,ek9pv0RRPr5puQ
4109,zkswap_finance,2025-02-07 00:00:00.000 UTC,1.062758e+06,1740275921.569736,NQnUQPwmG7EWFw
4110,zkswap_finance,2025-02-21 00:00:00.000 UTC,1.983659e+06,1740275921.569736,aPvv5TVsbhECZg
4111,zyberswap,2025-02-11 00:00:00.000 UTC,2.327945e+02,1740275921.569736,YItxYdzjY4NFjQ


In [4]:
# y2k_price_data
db.execute("select * from dune.y2k_price_data").fetchdf()

,timestamp,year,month,blockchain,contract_address,symbol,decimals,price,_dlt_load_id,_dlt_id
0,2022-09-17 00:00:00.000 UTC,2022,9,ethereum,0x4efdc844a4b860cd701844a7cf00bca610517647,Y2K,18,1.247929e-04,1740275921.569736,NIXWAswZfUGHGg
1,2022-10-09 00:00:00.000 UTC,2022,10,ethereum,0x4efdc844a4b860cd701844a7cf00bca610517647,Y2K,18,2.951592e-05,1740275921.569736,HlL2WypSBr/IJA
2,2022-10-13 00:00:00.000 UTC,2022,10,ethereum,0x4efdc844a4b860cd701844a7cf00bca610517647,Y2K,18,2.951592e-05,1740275921.569736,+49neAZSrQ6y2g
3,2022-10-29 00:00:00.000 UTC,2022,10,ethereum,0x4efdc844a4b860cd701844a7cf00bca610517647,Y2K,18,2.951592e-05,1740275921.569736,1R+7PZR5RcH5/g
4,2022-11-17 00:00:00.000 UTC,2022,11,ethereum,0x4efdc844a4b860cd701844a7cf00bca610517647,Y2K,18,2.951592e-05,1740275921.569736,mrdCVSfS40zBwg
...,...,...,...,...,...,...,...,...,...,...
4454,2024-12-20 00:00:00.000 UTC,2024,12,base,0xc644a35065c918182662c8349f2b39b23db27bc9,Y2K,18,1.404324e-07,1740275921.569736,4zkYqDVFMQTpJA
4455,2024-12-31 00:00:00.000 UTC,2024,12,arbitrum,0x65c936f008bc34fe819bce9fa5afd9dc2d49977f,Y2K,18,8.450507e-01,1740275921.569736,Zm6nMGi6FEelLw
4456,2025-01-04 00:00:00.000 UTC,2025,1,arbitrum,0x65c936f008bc34fe819bce9fa5afd9dc2d49977f,Y2K,18,6.620057e-01,1740275921.569736,H7oZqtAgeaeR0A
4457,2025-01-08 00:00:00.000 UTC,2025,1,arbitrum,0x65c936f008bc34fe819bce9fa5afd9dc2d49977f,Y2K,18,5.626881e-01,1740275921.569736,ygKm8I3ExwD+eQ


In [5]:
# y2k_price_data_no_pk (no primary key), data is append-only
db.execute("select * from dune.y2k_price_data_no_pk").fetchdf()

,timestamp,year,month,blockchain,contract_address,symbol,decimals,price,_dlt_load_id,_dlt_id
0,2022-12-19 00:00:00.000 UTC,2022,12,arbitrum,0x65c936f008bc34fe819bce9fa5afd9dc2d49977f,Y2K,18,2.421979,1740275921.569736,cSvxWKL7d6Fb4A
1,2022-12-20 00:00:00.000 UTC,2022,12,arbitrum,0x65c936f008bc34fe819bce9fa5afd9dc2d49977f,Y2K,18,0.706515,1740275921.569736,pTWxkS17CEUuYQ
2,2022-12-21 00:00:00.000 UTC,2022,12,arbitrum,0x65c936f008bc34fe819bce9fa5afd9dc2d49977f,Y2K,18,0.977897,1740275921.569736,ptPHlkNAt8ABJQ
3,2022-12-22 00:00:00.000 UTC,2022,12,arbitrum,0x65c936f008bc34fe819bce9fa5afd9dc2d49977f,Y2K,18,0.998779,1740275921.569736,nE/wFRqZJvrgeA
4,2022-12-23 00:00:00.000 UTC,2022,12,arbitrum,0x65c936f008bc34fe819bce9fa5afd9dc2d49977f,Y2K,18,0.934869,1740275921.569736,mq43U64uTs1Ghw
...,...,...,...,...,...,...,...,...,...,...
4454,2024-12-16 00:00:00.000 UTC,2024,12,ethereum,0xe71fca0818e256c7a570149217ff66afa47445ac,Y2K,18,0.000170,1740275921.569736,RhhAR9JNEA08dg
4455,2024-12-17 00:00:00.000 UTC,2024,12,ethereum,0xe71fca0818e256c7a570149217ff66afa47445ac,Y2K,18,0.000170,1740275921.569736,jVM7yYftw6QjcQ
4456,2024-12-18 00:00:00.000 UTC,2024,12,ethereum,0xe71fca0818e256c7a570149217ff66afa47445ac,Y2K,18,0.000170,1740275921.569736,c3U7mq85mfoLGw
4457,2024-12-19 00:00:00.000 UTC,2024,12,ethereum,0xe71fca0818e256c7a570149217ff66afa47445ac,Y2K,18,0.000170,1740275921.569736,QE0mAqKUE6CwXg


In [6]:
# custom_sql
db.execute("select * from dune.custom_sql").fetchdf()

,timestamp,blockchain,contract_address,symbol,price,_dlt_load_id,_dlt_id
0,2024-03-15 00:00:00.000 UTC,base,0x532f27101965dd16442e59d40670faf5ebb142e4,BRETT,0.041759,1740275921.569736,ejoktbK6gJHhjw
1,2024-03-16 00:00:00.000 UTC,base,0x532f27101965dd16442e59d40670faf5ebb142e4,BRETT,0.041509,1740275921.569736,NgNg+jjuJo34rQ
2,2024-04-23 00:00:00.000 UTC,base,0x532f27101965dd16442e59d40670faf5ebb142e4,BRETT,0.064760,1740275921.569736,f8oViEVMjocqtg
3,2024-05-02 00:00:00.000 UTC,base,0x532f27101965dd16442e59d40670faf5ebb142e4,BRETT,0.032190,1740275921.569736,gvLiq2QHSreq+A
4,2024-05-03 00:00:00.000 UTC,base,0x532f27101965dd16442e59d40670faf5ebb142e4,BRETT,0.037632,1740275921.569736,ZDeKT+RrfnVNYQ
...,...,...,...,...,...,...,...
359,2024-08-23 00:00:00.000 UTC,base,0x532f27101965dd16442e59d40670faf5ebb142e4,BRETT,0.096338,1740275921.569736,zcQdY13EJ9aTZg
360,2024-09-08 00:00:00.000 UTC,base,0x532f27101965dd16442e59d40670faf5ebb142e4,BRETT,0.070433,1740275921.569736,DDeUe8AyffpjQA
361,2024-09-12 00:00:00.000 UTC,base,0x532f27101965dd16442e59d40670faf5ebb142e4,BRETT,0.079640,1740275921.569736,7V/EPUTjtdF28g
362,2024-11-24 00:00:00.000 UTC,base,0x532f27101965dd16442e59d40670faf5ebb142e4,BRETT,0.163162,1740275921.569736,QxG/d9qcqWwT3g


In [7]:
# close the connection
db.close()